In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import grasp
from grasp import GrASP, CustomAttribute, remove_specialized_patterns
from sklearn.model_selection import train_test_split
from typing import Iterable, List, Set, Callable, Optional, Union, Sequence

## Load the data
- We use the **IBM Debater® - Evidence Sentences** dataset from the following paper which can be downloaded [here](https://www.research.ibm.com/haifa/dept/vst/debating_data.shtml#Argument%20Detection). (Please select the ACL 2018 dataset)
```
Shnarch, E., Alzate, C., Dankin, L., Gleize, M., Hou, Y., Choshen, L., ... & Slonim, N. (2018, July). Will it blend? blending weak and strong labeled data in a neural network for argumentation mining. In Proceedings of the 56th Annual Meeting of the Association for Computational Linguistics (Volume 2: Short Papers) (pp. 599-605).
```

- Load the data

In [4]:
import pandas as pd
def get_data(split = 'train'):
    df = pd.read_csv(f'data/IBMDebaterEvidenceSentences/{split}.csv')
    texts = df['candidate masked'].tolist()
    labels = list(map(int, df['label'].tolist()))
    return texts, labels

In [6]:
X_train, y_train = get_data(split = 'train')
positive = [t for idx, t in enumerate(X_train) if y_train[idx]]
negative = [t for idx, t in enumerate(X_train) if not y_train[idx]]
print(len(positive), len(negative))

1499 2566


## (Optional) Add a custom attribute
This attribute checks whether a word is in a given list of argumentative words or not.

In [7]:
try:
    ARGUMENTATIVE_LEXICON = [line.strip().lower() for line in open('data/argumentative_unigrams_lexicon_shortlist.txt', 'r') if line.strip() != '']
    def _argumentative_extraction(text: str, tokens: List[str]) -> List[Set[str]]:
        tokens = map(str.lower, tokens)
        ans = []
        for t in tokens:
            t_ans = []
            if t.lower() in ARGUMENTATIVE_LEXICON:
                t_ans.append('Yes')
            ans.append(set(t_ans))
        return ans

    def _argumentative_translation(attr:str, 
                          is_complement:bool = False) -> str:
        word = attr.split(':')[1]
        assert word == 'Yes'
        return 'an argumentative word'

    ArgumentativeAttribute = CustomAttribute(name = 'ARGUMENTATIVE', extraction_function = _argumentative_extraction, translation_function = _argumentative_translation)

except:
    ArgumentativeAttribute = None

print(ArgumentativeAttribute)

ARGUMENTATIVE


## Run GrASP

In [8]:
# Create the GrASP engine
if ArgumentativeAttribute is not None:
    grasp_model = GrASP(gaps_allowed = 2, num_patterns = 100, include_standard = ['LEMMA', 'POS', 'NER', 'HYPERNM', 'SENTIMENT'],
                        include_custom = [ArgumentativeAttribute],
                        correlation_threshold = 0.5, alphabet_size = 100)
else:
    grasp_model = GrASP(gaps_allowed = 2, num_patterns = 100, include_standard = ['LEMMA', 'POS', 'NER', 'HYPERNM', 'SENTIMENT'],
                        correlation_threshold = 0.5, alphabet_size = 100)

In [9]:
# Fit GrASP to the dataset
the_patterns = grasp_model.fit_transform(positive, negative)

  0%|          | 6/1499 [00:00<00:28, 52.77it/s]

Step 1: Create augmented texts


100%|██████████| 2566/2566 [00:48<00:00, 52.92it/s]


Step 2: Find frequent attributes


  1%|          | 6/742 [00:00<00:17, 42.66it/s]

...

Length 4 / 5; New candidates = 6548
Finding top k: 10 / 100
Finding top k: 20 / 100
Finding top k: 30 / 100
Finding top k: 40 / 100
Finding top k: 50 / 100
Finding top k: 60 / 100
Finding top k: 70 / 100
Finding top k: 80 / 100
Finding top k: 90 / 100


  0%|          | 0/9 [00:00<?, ?it/s]

Finding top k: 100 / 100
Example of current patterns
Pattern: [['LEMMA:that', 'SPACY:POS-ADP']]
Window size: 3
Class: Positive
Precision: 0.546
Match: 1239 (30.5%)
Gain = 0.042
Metric (global) = 0.042
Examples ~ Class Positive:
[MATCH]: In August 2008 , a group of college presidents calling itself the Amethyst Initiative asserted that:['SPACY:POS-ADP', 'LEMMA:that'] lowering TOPIC_CONCEPT to 18 ( presumably ) was one way to curb the " culture of dangerous binge drinking " among college students [ REF ] . 
-------------------------
[MATCH]: The Washington Post reported in 2003 that:['SPACY:POS-ADP', 'LEMMA:that'] Lois Boland ( USPTO Director of International Relations ) said " that TOPIC_CONCEPT runs counter to the mission of WIPO , which is to promote intellectual - property rights . " 
-------------------------
Counterexamples ~ Not class Positive:
[MATCH]: The lawsuit notes that:['SPACY:POS-ADP', 'LEMMA:that'] Bedier knew all of PayPal 's future plans for TOPIC_CONCEPT , as well as a

100%|██████████| 9/9 [00:06<00:00,  1.34it/s]


Length 5 / 5; New candidates = 1788
Finding top k: 10 / 100
Finding top k: 20 / 100
Finding top k: 30 / 100
Finding top k: 40 / 100
Finding top k: 50 / 100
Finding top k: 60 / 100
Finding top k: 70 / 100
Finding top k: 80 / 100
Finding top k: 90 / 100
Finding top k: 100 / 100
Example of current patterns
Pattern: [['LEMMA:that', 'SPACY:POS-ADP']]
Window size: 3
Class: Positive
Precision: 0.546
Match: 1239 (30.5%)
Gain = 0.042
Metric (global) = 0.042
Examples ~ Class Positive:
[MATCH]: Sullivan is a vocal critic of the Environmental Protection Agency , claiming that:['SPACY:POS-ADP', 'LEMMA:that'] TOPIC_CONCEPT being pushed by the Obama Administration are harmful to the U.S. economy . 
-------------------------
[MATCH]: The Anti - Defamation League has stated that:['SPACY:POS-ADP', 'LEMMA:that'] " TOPIC_CONCEPT is a contemporary form of the classic anti - Semitic doctrine of the evil , manipulative and threatening world Jewish conspiracy"[REF ] and French historian Valérie Igounet has wr

In [10]:
# Print the learned patterns
for idx, p in enumerate(the_patterns):
    print(f'Rank {idx+1}')
    print(p)

Rank 1
Pattern: [['LEMMA:that', 'SPACY:POS-ADP']]
Window size: 3
Class: Positive
Precision: 0.546
Match: 1239 (30.5%)
Gain = 0.042
Metric (global) = 0.042
Examples ~ Class Positive:
[MATCH]: Zeigler believed Riley should be disciplined to say " Sir " and " Ma'am " to adults , and that:['SPACY:POS-ADP', 'LEMMA:that'] TOPIC_CONCEPT was the best means of disciplining a child . 
-------------------------
[MATCH]: Researchers found that:['SPACY:POS-ADP', 'LEMMA:that'] a removal of TOPIC_CONCEPT policies from all colleges and universities would result in a significant drop in minority presence in all institutions of higher education , upholding TOPIC_CONCEPT role in diversity in the American classroom [ REF ] . 
-------------------------
Counterexamples ~ Not class Positive:
[MATCH]: In a column published in the New York Times on June 15 , 2011 Kristof argued that:['SPACY:POS-ADP', 'LEMMA:that'] the United States military was a prime example of how comprehensive social safety net , TOPIC_CON

In [12]:
print(f'  #    class Cov(%)    Prec    Gain    Pattern')
for idx, p in enumerate(the_patterns):
    print(f'{idx+1:>3} {p.support_class}   {round(p.coverage*100, 1):>4}   {p.precision:.3f}   {p.metric:.3f}    {p.get_pattern_id()}')

  #    class Cov(%)    Prec    Gain    Pattern
  1 Positive   30.5   0.546   0.042    [['LEMMA:that', 'SPACY:POS-ADP']]
  2 Negative   42.7   0.546   0.017    [['LEMMA:]']]
  3 Negative   27.7   0.518   0.015    [['LEMMA:that'], ['SPACY:POS-VERB']]
  4 Positive    8.9   0.579   0.013    [['ARGUMENTATIVE:Yes'], ['LEMMA:that']]
  5 Negative   26.6   0.529   0.012    [['SPACY:POS-VERB'], ['SPACY:POS-ADP'], ['SPACY:POS-VERB']]
  6 Positive    5.2   0.627   0.011    [['LEMMA:study']]
  7 Positive    2.5   0.728   0.010    [['LEMMA:risk']]
  8 Negative   27.6   0.541   0.009    [['ARGUMENTATIVE:Yes'], ['SPACY:POS-ADP']]
  9 Positive    4.9   0.607   0.009    [['LEMMA:ban']]
 10 Positive    4.8   0.607   0.009    [['ARGUMENTATIVE:Yes'], ['ARGUMENTATIVE:Yes']]
 11 Positive   10.5   0.525   0.009    [['LEMMA:that', 'SPACY:POS-ADP'], ['SPACY:POS-DET']]
 12 Positive    6.0   0.578   0.008    [['LEMMA:energy']]
 13 Positive    7.7   0.546   0.008    [['ARGUMENTATIVE:Yes'], ['SPACY:POS-ADP'], ['SPA

## Save the patterns to a json file
We can use this json file as an input of the web demo tool for exploring the learned patterns and the training data

In [13]:
grasp_model.to_json('results/case_study_2.json')

100%|██████████| 100/100 [00:00<00:00, 231.03it/s]


Successfully dump the results to results/case_study_2.json
